# Machine Learning I - Project 1
## Group Members: Michael Butros, June Claiborne, Lila Ghemri

## Section 1: Loading Libraries

In [1]:
import numpy as np
from numpy import random
from csv import reader
from random import randrange
from random import seed
import pandas as pd
import colorama
from colorama import Fore
from math import sqrt
from math import exp
from math import pi

### Read in dataset, add sum, and score columns

In [2]:
df = pd.read_csv("MLProject1.csv")
print(df.head(3))
df['Sum'] = df.iloc[: , 1:].sum(axis=1)
df['Score'] = df.Sum.div(17).round(4)*100
df['GoodLocation']=0
print("\n")
for ind in df.index:
      if df['Score'][ind] >=60:
            df['GoodLocation'][ind]=1
            print(Fore.GREEN + df['Location'][ind]+ " is a good location with %.2f "  % round(df['Score'][ind],2))
      else: 
          print(Fore.BLACK + df['Location'][ind]+ " is not a good location with %.2f"  % round(df['Score'][ind],2))
          df['GoodLocation'][ind]=0

         Location  Cost  Growth Potential  Crime Rate  Visibility  Parking  \
0          Austin     0                 1           1           1        0   
1  Salt Lake City     1                 0           1           0        0   
2         Raleigh     1                 1           1           1        1   

   Near Target Community  Access to Public Transportation  Permits Required  \
0                      0                                0                 0   
1                      0                                1                 1   
2                      0                                0                 0   

   Zoning Laws  Insurance Needed  Outdoor Play Area  Grooming Services  \
0            1                 1                  1                  0   
1            1                 1                  1                  1   
2            1                 0                  1                  1   

   Population Age >65  Retired  Median Income of Population  \
0         

/var/folders/l7/qkjnbb616nzd10h2cxfy7nj00000gp/T/ipykernel_35921/4061320473.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['GoodLocation'][ind]=0
/var/folders/l7/qkjnbb616nzd10h2cxfy7nj00000gp/T/ipykernel_35921/4061320473.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['GoodLocation'][ind]=1


### Printing few rows of the dataset

In [3]:
print(df.head(3))

         Location  Cost  Growth Potential  Crime Rate  Visibility  Parking  \
0          Austin     0                 1           1           1        0   
1  Salt Lake City     1                 0           1           0        0   
2         Raleigh     1                 1           1           1        1   

   Near Target Community  Access to Public Transportation  Permits Required  \
0                      0                                0                 0   
1                      0                                1                 1   
2                      0                                0                 0   

   Zoning Laws  ...  Outdoor Play Area  Grooming Services  Population Age >65  \
0            1  ...                  1                  0                   1   
1            1  ...                  1                  1                   1   
2            1  ...                  1                  1                   0   

   Retired  Median Income of Population  Pric

## Section 2: Baseline Algorithms (Random Prediction and Zero Rule)

In [4]:
from numpy import array
# Calculate accuracy percentage between two lists
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

def random_algorithm(train,test):
  output_values = [row[-1] for row in train]
  unique = list(set(output_values))
  predicted = list()
  for _ in test:
    index = randrange(len(unique))
    predicted.append(unique[index])
  return predicted

# zero rule algorithm for classification
def zero_rule_algorithm_classification(train, test):
    output_values = [row[-1] for row in train]
    prediction = max(set(output_values), key=output_values.count)
    predicted = [prediction for i in range(len(test))]
    return predicted

seed(2023)
#setting up the attribute format to work with algorithms above
GL=array(df['GoodLocation'])
new_lst = [[i] for i in GL]

xs = [None] * len(df)
new_xs=[[i] for i in xs]

predictions=random_algorithm(new_lst,new_xs)
print("Predicted Values using Random", predictions)

# Test accuracy
accuracy = accuracy_metric(GL, predictions)
print("Accuracy of random algorithm ", accuracy,"%")
predictions = zero_rule_algorithm_classification(new_lst,new_xs)
print("Predicted Values using Zero rule",  predictions)
accuracy = accuracy_metric(GL, predictions)
print("Accuracy of zero rule algorithm", accuracy, "%")

Predicted Values using Random [1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1]
Accuracy of random algorithm  57.99999999999999 %
Predicted Values using Zero rule [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Accuracy of zero rule algorithm 74.0 %


## Section 3: The Perceptron Algorithm 

In [5]:
#Using Perceptron to make predictions
import numpy as np
from  random import   seed
from  random import   randrange 
from  csv  import   reader

#Cleaning the dataset:

def clean_data(dataset):
    df1= dataset.drop(columns=["Location","Sum", "Score"])
    return df1

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    #print(dataset)
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for _ in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    #print(dataset_split)
    return dataset_split

def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

# Evaluate  an algorithm using  a cross  validation split
def  evaluate_algorithm(dataset, algorithm,  n_folds, *args):
    folds = cross_validation_split(dataset,  n_folds)
   # print(folds)
    scores  = list()
    for fold in folds: 
        train_set =  list(folds) 
        train_set.remove(fold) 
        train_set = sum(train_set,  []) 
        test_set = list()
        for row in fold: 
            row_copy =  list(row) 
            test_set.append(row_copy) 
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy  = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return  scores

# Make a prediction with weights
def predict(row, weights):
    activation = weights[0]
    for i in range(len(row)-1):
        activation += weights[i + 1] * row[i]
    return 1.0 if activation >= 0.0 else 0.0

# Estimate Perceptron weights using stochastic gradient descent
def train_weights(train, l_rate, n_epoch):
    weights = [0.0 for i in range(len(train[0]))]
    for _ in range(n_epoch):
        for row in train:
            prediction = predict(row, weights)
            error = row[-1] - prediction
            weights[0] = weights[0] + l_rate * error
            for i in range(len(row)-1):
                weights[i + 1] = weights[i + 1] + l_rate * error * row[i]
    print("Weights for each fold:")
    print(weights)
    return weights
# Perceptron Algorithm With Stochastic Gradient Descent
def perceptron(train, test, l_rate, n_epoch):
    predictions = list()
    weights = train_weights(train, l_rate, n_epoch)
    for row in test:
        prediction = predict(row, weights)
        predictions.append(prediction)
    return(predictions)
           
    
# Test the Perceptron algorithm on the Doggy Hotel  dataset
seed(2023)
# prepare data
df1=clean_data(df)
dlist = df1.values.tolist()
n_folds = 3
l_rate = 0.01
n_epoch = 500
scores = evaluate_algorithm(dlist, perceptron, n_folds, l_rate, n_epoch)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Weights for each fold:
[-0.36000000000000015, 0.010000000000000004, 0.05000000000000001, 0.04000000000000001, 0.09, -0.01, 0.04, 0.030000000000000006, 0.060000000000000005, 0.07, -0.04000000000000001, -0.060000000000000005, 0.12999999999999998, -0.010000000000000004, 0.010000000000000004, 0.03, -3.469446951953614e-18, 0.060000000000000005]
Weights for each fold:
[-0.10999999999999999, 0.009999999999999997, -0.009999999999999997, -0.02, 0.01, 0.009999999999999997, -0.019999999999999997, 0.03, 0.0, 0.07, -0.02, -0.07, 0.04, 0.019999999999999997, 0.009999999999999997, 0.030000000000000006, 0.02, -0.03]
Weights for each fold:
[-0.12999999999999998, 0.01, 0.0, 0.03, 0.019999999999999997, 0.04, 0.010000000000000004, 0.05, 0.060000000000000005, -0.020000000000000004, -0.03, -0.05, 0.009999999999999997, 0.07, -0.009999999999999997, -0.03, 0.01, -0.019999999999999997]
Scores: [75.0, 68.75, 50.0]
Mean Accuracy: 64.583%


## Section 4: The Naive Bayes Algorithm 